In [285]:
import _csv
import math
import re
import gzip
import json
import numpy as np
import cohere
from cohere.classify import Example
co = cohere.Client('Odj4ClpHI3fobuB9bErJkfzlszJwQ018Z99gUAyE')
KEY = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [286]:
def print_classifications(response, data=True):
    results = {KEY[i]: [] for i in range(len(KEY))}
    for i in range(len(response.classifications)):
        prediction = int(response.classifications[i].prediction)
        confidence = response.classifications[i].confidence
        if confidence >= 0.7:
            results[KEY[prediction]].append(confidence)
        if data:
            data = response.classifications[i].input
            print(f'Input: {data}')
        print(f'Class: {prediction}, Confidence: {confidence} \n')
    return results

In [287]:
def feedback(results):
    print(results, '\n')
    evaluation = []
    for i in range(len(KEY)):
        data = np.array(results[KEY[i]])
        if len(data) == 0:
            data = [0]
        average = np.mean(data)
        n = len(data)
        coefficient = n / (2 + n)
        evaluation.append(coefficient * average)
    print(evaluation)
    return KEY[np.argmax(evaluation)]

In [288]:
user_prompt1 = """
My son claims that hes been having extremely violent thoughts and dreams. Not violent like he's hurting someone, but violent thoughts like horrible things happening to his loved ones. He explained one of his dreams the other day and it was so violent it was sickening. It was far beyond anything in a horror movie, he says he can't help these thoughts they just pop up. Please help!
""".strip()

user_prompt2 = """
I went to the gym last night and saw many people with good fitness. However I am fat and obese. So I couldn't stand being there and came home while cry
""".strip()

user_prompt3 = """
I did not get any valentine chocolate today which makes me very sad. Man I want a girlfriend quickly.
""".strip()

user_prompt4 = """
I came home and suddenly, boom!, happy birth day. My parents really surprised day. What a lovely day it was!
""".strip()

user_prompt5 = """
It's important to note that IMDb ratings are not an absolute measure of a movie's quality, but rather a reflection of the collective opinion of IMDb users who have rated the movie.
""".strip()

user_prompt = """
Why the hell did you shoot at that stranger. You killed him bastard. Now you go to jail.
""".strip()

In [289]:
# prompt = f"""
# Break the paragraph into small sentences {user_prompt}
# """
#
# summary = co.generate(
#     model='command-xlarge-nightly',
#     prompt= prompt,
#     max_tokens=100,
#     temperature=0.9,
#     stop_sequences=["--"])
# print(summary)

In [290]:
cut_user_prompt = re.split(r'[,.!?-]', user_prompt.replace(' and ', '.'))
cut_user_prompt = list(filter(None, cut_user_prompt))
print(cut_user_prompt)

response = co.classify(
  model='0c667180-ef42-4c4d-aee8-e6fa29177e43-ft',
  inputs=cut_user_prompt)

results = print_classifications(response)
print(feedback(results))

['Why the hell did you shoot at that stranger', ' You killed him bastard', ' Now you go to jail']
Input: Why the hell did you shoot at that stranger
Class: 3, Confidence: 0.85519105 

Input:  You killed him bastard
Class: 3, Confidence: 0.50845504 

Input:  Now you go to jail
Class: 4, Confidence: 0.58861053 

{'sadness': [], 'joy': [], 'love': [], 'anger': [0.85519105], 'fear': [], 'surprise': []} 

[0.0, 0.0, 0.0, 0.2850636833333333, 0.0, 0.0]
anger
